### Project - Airline AI Assistant
We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
import os, json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

MODEL = "gpt-4.1-mini"

In [4]:
system_message = """
You are a helpful assistant, for an airline called FlightAI.
Give short and courteous responses no more than one sentence long.
Always be accurate, if you don't know the answer, just say you don't know.
"""

In [6]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


#### Tools

- we see that it does not have some flight related information
- to augment this unknown information, we can add tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

##### Step 1: Setting up the tool

In [7]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [8]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $799'

In [9]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## this is what is sent to LLM
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

##### Step 2: Getting LLM to use our Tool
There's some fiddly stuff to allow LLM "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [11]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if the finish reason is tool calls then, we need to call the tool
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        for message in messages:
            print(message)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [12]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7889
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
{'role': 'system', 'content': "\nYou are a helpful assistant, for an airline called FlightAI.\nGive short and courteous responses no more than one sentence long.\nAlways be accurate, if you don't know the answer, just say you don't know.\n"}
{'role': 'user', 'content': 'hi'}
{'role': 'assistant', 'content': 'Hello! How can I assist you with your flight today?'}
{'role': 'user', 'content': 'flight to london price'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_Fk5XnHgwsCx9fLbqqJTqXjfQ', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
{'role': 'tool', 'content': 'The price of a ticket to London is $799', 'tool_call_id': 'call_Fk5XnHgwsCx9fLbqqJTqXjfQ'}


#### Let's make a couple of improvements
Handling multiple tool calls in 1 response

Handling multiple tool calls 1 after another

In [14]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if the finish reason is tool calls then, we need to call the tool
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.append(response)

        for message in messages:
            print(message)

        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [15]:
def handle_tool_calls(message):
    response = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            response.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


Tool called for city Paris
Tool called for city London
{'role': 'system', 'content': "\nYou are a helpful assistant, for an airline called FlightAI.\nGive short and courteous responses no more than one sentence long.\nAlways be accurate, if you don't know the answer, just say you don't know.\n"}
{'role': 'user', 'content': 'flight price for paris and london'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_XXcqlcOd6dMDeAUuO5ZVXLzG', function=Function(arguments='{"destination_city": "Paris"}', name='get_ticket_price'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_jEqi8nCwfZSyNGpb4DgO7YXe', function=Function(arguments='{"destination_city": "London"}', name='get_ticket_price'), type='function')])
{'role': 'tool', 'content': 'The price of a ticket to Paris is $899', 'tool_call_id': 'call_XXcqlcOd6dMDeAUuO5ZVXLzG'}
{'role': 'tool', 'content': 'Th

If we ask "Check the price for London, only if the price is less than $1000 then check price for Tokyo", It will only make one call to London and ends there, because we do not have loop in tool call check.

In [17]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if the finish reason is tool calls then, we need to call the tool
    while response.choices[0].finish_reason=="tool_calls":
        message_asst = response.choices[0].message
        tool_response = handle_tool_calls(message_asst)
        messages.append(message_asst)
        messages.extend(tool_response)
 
        for msg in messages:
            print(msg)

        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        print(f"Final response: {response.choices[0].message}")  # 
    
    return response.choices[0].message.content